In [1]:
import sklearn

In [2]:
import tensorflow

In [2]:
import pandas as pd
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
import Plotting
import Tools
from joblib import dump, load

def drop_neg(value):
    return value == -999.0

def get_class(value,cls):
    return value != cls

In [63]:
cols = 'DER_mass_MMC|DER_mass_transverse_met_lep|DER_pt_h|DER_deltar_tau_lep|DER_mass_vis|Label|Weight'
seed = 7
test_size = 0.33

def get_dataset(filename):
    return pd.read_csv(filename)

def select_columns(df):
    return df.filter(regex=cols)

def get_label_ints(df):
    return np.where(df['Label'] == 's',1,0)
    
def get_test_data(df):
    return df.drop('Label',axis=1).drop('Weight',axis=1).values

def train():
    training_set = get_dataset("train.csv")
    d_train = select_columns(training_set)
    X_train = get_test_data(d_train)
    Y_train = get_label_ints(d_train)
    # split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
    # fit model no training data
    model = XGBClassifier()
    model.fit(X_train, y_train)
    # make predictions for test data
    y_pred = model.predict(X_test)
    return model

def validate(model):
    test_set = get_dataset("validation.csv")
    d_test = select_columns(test_set)
    X_valid = d_test.values

    y_test = model.predict(X_valid)
    valid_labels = test_set['EventId'].values
    output = []
    for i in range(len(y_test)):
        output.append( (valid_labels[i],y_test[i]))

    sorted_output = sorted(output,key=lambda x: -x[1])

    with open('validation_submission.csv','w') as f:
        f.write('EventId,RankOrder,Class\n')
        for i in range(len(sorted_output)):
            eventID = sorted_output[i][0]
            value = sorted_output[i][0]
            cls = 's' if sorted_output[i][1] == 1 else 'b'
            f.write("{},{},{}\n".format(eventID,i+1,cls))
        f.close()

def test(model):
    test_set = get_dataset("test.csv")
    d_test = select_columns(test_set)
    X_valid = d_test.values

    y_test = model.predict(X_valid)
    valid_labels = test_set['EventId'].values
    output = []
    for i in range(len(y_test)):
        output.append( (valid_labels[i],y_test[i]))

    sorted_output = sorted(output,key=lambda x: -x[1])

    with open('test_submission.csv','w') as f:
        f.write('EventId,RankOrder,Class\n')
        for i in range(len(sorted_output)):
            eventID = sorted_output[i][0]
            value = sorted_output[i][0]
            cls = 's' if sorted_output[i][1] == 1 else 'b'
            f.write("{},{},{}\n".format(eventID,i+1,cls))
        f.close()
    
model = train()
validate(model)
test(model)